In [1]:
import pandas as pd
data_pol=pd.read_csv('C:/Users/Usuario/Documents/Men_argue-Nature_acts/CSV/pollution_pop.csv')
data_city=pd.read_csv('C:/Users/Usuario/Documents/Men_argue-Nature_acts/CSV/cities_database.csv')

data_pol.columns = data_pol.columns.str.strip()

columns_to_convert = ['pm25', 'pm10', 'o3', 'no2', 'so2', 'co', 'year', 'pop']
data_pol[columns_to_convert] = data_pol[columns_to_convert].apply(pd.to_numeric, errors='coerce')

columns_to_fill = ['pm25', 'pm10', 'o3', 'no2', 'so2']

for column in columns_to_fill:
    mean_value = data_pol[column].mean()
    data_pol[column].fillna(mean_value, inplace=True)
data_pol['co'] = data_pol['co'].fillna(0)
data_pol.columns
def get_city_data(madrid):
    import requests
    import pandas as pd
    import json
    
    api_url = 'https://api.api-ninjas.com/v1/city?name={}'.format(city_name)
    response = requests.get(api_url, headers={'X-Api-Key': 'XhKog+U7kqeVm1jW7MeNDg==ZsJmH2sbNbEnRq3H'})
    if response.status_code == requests.codes.ok:
        response_text = response.text
        data = json.loads(response_text)
        df = pd.DataFrame(data)
        return df
    else:
        print("Error:", response.status_code, respoonse_text)
        return None
city_name = 'madrid'
city_df = get_city_data(city_name)
if city_df is not None:
    print(city_df)
city_df.drop(columns='population',axis=1,inplace=True)

data_pol = data_pol.assign(key=1)
city_df = city_df.assign(key=1)


result = pd.merge(city_df, data_pol, on='key').drop('key', axis=1)
result = result.rename(columns={'pop': 'population', 'pm25': 'pm2.5'})
new_order = ['name', 'latitude', 'longitude', 'country', 'population', 'is_capital', 'co', 'no2', 'o3', 'so2', 'pm2.5', 'pm10', 'overall_aqi']
result = result.reindex(columns=new_order)
result['overall_aqi'].fillna(0,inplace=True)

     name  latitude  longitude country  population  is_capital
0  Madrid   40.4189    -3.6919      ES     3266126        True


In [2]:
result

,name,latitude,longitude,country,population,is_capital,co,no2,o3,so2,pm2.5,pm10,overall_aqi
0,Madrid,40.4189,-3.6919,ES,6044000,True,0.0,18.0,20.0,2.000000,55.805431,13.0,0.0
1,Madrid,40.4189,-3.6919,ES,6132000,True,0.0,26.0,60.0,2.000000,73.000000,39.0,0.0
2,Madrid,40.4189,-3.6919,ES,6132000,True,0.0,12.0,42.0,2.000000,69.000000,21.0,0.0
3,Madrid,40.4189,-3.6919,ES,6132000,True,0.0,17.0,58.0,2.000000,36.000000,21.0,0.0
4,Madrid,40.4189,-3.6919,ES,6132000,True,0.0,26.0,53.0,3.000000,40.000000,46.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3589,Madrid,40.4189,-3.6919,ES,6783000,True,0.0,11.0,28.0,3.105897,43.000000,9.0,0.0
3590,Madrid,40.4189,-3.6919,ES,6783000,True,0.0,22.0,19.0,3.105897,45.000000,13.0,0.0
3591,Madrid,40.4189,-3.6919,ES,6783000,True,0.0,20.0,17.0,3.105897,62.000000,16.0,0.0
3592,Madrid,40.4189,-3.6919,ES,6783000,True,0.0,11.0,31.0,3.105897,41.000000,13.0,0.0


In [7]:
def get_city_data(city_name):
    import requests
    import pandas as pd
    import json

    api_url = 'https://api.api-ninjas.com/v1/city?name={}'.format(city_name)
    response = requests.get(api_url, headers={'X-Api-Key': 'XhKog+U7kqeVm1jW7MeNDg==ZsJmH2sbNbEnRq3H'})
    if response.status_code == requests.codes.ok:
        response_text = response.text
        data = json.loads(response_text)
        df = pd.DataFrame(data)
        return df
    else:
        print("Error:", response.status_code, response_text)
        return None

# For testing:
if __name__ == "__main__":
    city_name = input('Enter the city name: ')
    city_df = get_city_data(city_name)
    if city_df is not None:
        print(city_df)


        name  latitude  longitude country  population  is_capital
0  Barcelona   41.3825     2.1769      ES     4588000       False


In [8]:
city_df

,name,latitude,longitude,country,population,is_capital
0,Barcelona,41.3825,2.1769,ES,4588000,False


In [10]:
def get_air_quality_data(city):
    import pandas as pd
    import requests
    api_url = 'https://api.api-ninjas.com/v1/airquality?city={}'.format(city)
    response = requests.get(api_url, headers={'X-Api-Key': 'XhKog+U7kqeVm1jW7MeNDg==ZsJmH2sbNbEnRq3H'})
    if response.status_code == requests.codes.ok:
        air_quality_data = response.json()
        if 'error' in air_quality_data:
            raise ValueError("Error: City not found in the database")
        else:
            data = {k.lower(): [v['concentration'], v['aqi']] for k, v in air_quality_data.items() if k != 'overall_aqi'}
            data['overall_aqi'] = [None, air_quality_data['overall_aqi']]
            pollution_df = pd.DataFrame(data)
            
            # Fill NaN values with 0
            pollution_df = pollution_df.fillna(0)
            
            return pollution_df
    else:
        raise requests.ConnectionError(f"Error: {response.status_code} - {response.text}")

# For testing:
if __name__ == "__main__":
    city_name = input("Enter a city name: ")
    try:
        pollution_df = get_air_quality_data(city_name)
        print(pollution_df)
    except ValueError as ve:
        print(ve)
    except requests.ConnectionError as ce:
        print(ce)

      co    no2     o3   so2  pm2.5  pm10  overall_aqi
0  317.1  18.68  35.41  0.97   3.27  5.35          0.0
1    3.0  23.00  30.00  1.00  10.00  4.00         30.0


In [12]:
pollution_df['city'] = city_name
city_df['city'] = city_name

In [13]:
joined_df = pd.merge(city_df, pollution_df, on='city', how='outer')

 
joined_df = joined_df.drop(columns='city',axis=1)

In [14]:
joined_df

,name,latitude,longitude,country,population,is_capital,co,no2,o3,so2,pm2.5,pm10,overall_aqi
0,Barcelona,41.3825,2.1769,ES,4588000,False,317.1,18.68,35.41,0.97,3.27,5.35,0.0
1,Barcelona,41.3825,2.1769,ES,4588000,False,3.0,23.00,30.00,1.00,10.00,4.00,30.0


In [15]:
def make_predictions_for_df(joined_df, num_predictions, model_path):
    from sklearn.ensemble import RandomForestRegressor
    import pandas as pd
    import joblib
    
    model = joblib.load(model_path)
    expanded_df = pd.concat([joined_df] * (num_predictions // len(joined_df) + 1), ignore_index=True)
    expanded_df = expanded_df.iloc[:num_predictions]
    predictions = model.predict(expanded_df)
    return predictions

In [17]:
# Remove the incompatible features from the dataframe used for predictions
joined_df = joined_df.drop(['latitude', 'longitude', 'name','country', 'is_capital', 'population], axis=1, errors='ignore')

In [21]:
joined_df = joined_df.drop(['country', 'is_capital', 'population'], axis=1, errors='ignore')

In [22]:
make_predictions_for_df(joined_df, 20, 'C:/Users/Usuario/Documents/Men_argue-Nature_acts/MODEL/pred_model.joblib')

array([[ 0.        , 20.79977044, 35.67088103,  3.47388713, 20.38      ,
         3.47388713,  0.        ],
       [ 0.        , 20.71825308, 28.32416386,  3.26694356, 18.24      ,
         3.26694356,  0.        ],
       [ 0.        , 20.79977044, 35.67088103,  3.47388713, 20.38      ,
         3.47388713,  0.        ],
       [ 0.        , 20.71825308, 28.32416386,  3.26694356, 18.24      ,
         3.26694356,  0.        ],
       [ 0.        , 20.79977044, 35.67088103,  3.47388713, 20.38      ,
         3.47388713,  0.        ],
       [ 0.        , 20.71825308, 28.32416386,  3.26694356, 18.24      ,
         3.26694356,  0.        ],
       [ 0.        , 20.79977044, 35.67088103,  3.47388713, 20.38      ,
         3.47388713,  0.        ],
       [ 0.        , 20.71825308, 28.32416386,  3.26694356, 18.24      ,
         3.26694356,  0.        ],
       [ 0.        , 20.79977044, 35.67088103,  3.47388713, 20.38      ,
         3.47388713,  0.        ],
       [ 0.        , 20.7182